In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=8c172d3117f7105928891bfd33483735cbef8d949f31b99d34bf57eecc499dd0
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Conv1D, GlobalMaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Sequential




import seaborn as sns
import matplotlib.pyplot as plt

import re
import glob
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import warnings
from wordcloud import STOPWORDS, WordCloud
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Datasets Sem-6/IR Datasets/Project/liar_dataset.zip'

Archive:  /content/drive/MyDrive/Datasets Sem-6/IR Datasets/Project/liar_dataset.zip
  inflating: README                  
  inflating: test.tsv                
  inflating: train.tsv               
  inflating: valid.tsv               


In [12]:
train_df = pd.read_csv('/content/drive/MyDrive/liar_dataset/train.tsv',sep='\t', header = None)
test_df = pd.read_csv('/content/drive/MyDrive/liar_dataset/test.tsv',sep='\t', header = None)
val_df = pd.read_csv('/content/drive/MyDrive/liar_dataset/valid.tsv',sep='\t', header = None)

In [13]:
train_df = train_df.drop([0, 8, 9, 10, 11, 12], axis = 1)
test_df = test_df.drop([0, 8, 9, 10, 11, 12], axis = 1)
val_df = val_df.drop([0, 8, 9, 10, 11, 12], axis = 1)

In [14]:
train_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']
test_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']
val_df.columns = ['label', 'statement', 'subject', 'speaker', 'speaker job title', 'state info', 'party affilation', 'location of statement']

In [15]:
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True)
test_df = test_df.dropna()
test_df = test_df.reset_index(drop=True)
val_df = val_df.dropna()
val_df = val_df.reset_index(drop=True)

In [16]:
train_df = pd.concat([train_df, val_df])
train_df = train_df.reset_index()

In [17]:
print('Training Data Dimensions -', train_df.shape)
print('Testing Data Dimensions -', test_df.shape)

Training Data Dimensions - (7585, 9)
Testing Data Dimensions - (853, 8)


In [18]:
labels_dict = {'mostly-true':4,'barely-true':2,'half-true':3,'false':1, 'true':5,'pants-fire':0}
train_df['label'] = train_df['label'].apply(lambda x: labels_dict[x])
test_df['label'] = test_df['label'].apply(lambda x: labels_dict[x])

In [19]:
def preprocess(text):
  text = text.lower() # lower - casing the text
  text = re.sub('<[^>]*>', ' ', text)
  text = re.sub('[\W]+', ' ', text)
  tokenizer = TreebankWordTokenizer()
  words = tokenizer.tokenize(text)
  text = ' '.join(words)
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w in stop_words] # removal of stopwords
  text = ' '.join(filtered_sentence)
  return text

In [20]:
# Standard Model['label' --> Target , 'statement' --> input text data]
def get_XY(data):
  y = data['label'].values
  X = data['statement'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

In [ ]:
# Preprocessing text data
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getXY(train_df)
X_test, y_test = getXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)



Epoch 1/10
190/190 [==============================] - 15s 71ms/step - loss: -2072.8821 - accuracy: 0.1938 - val_loss: -10735.0938 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 16s 83ms/step - loss: -80415.0312 - accuracy: 0.1943 - val_loss: -208881.4062 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 9s 46ms/step - loss: -606200.1250 - accuracy: 0.1943 - val_loss: -1132892.5000 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 11s 60ms/step - loss: -2323729.0000 - accuracy: 0.1943 - val_loss: -3617297.5000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 11s 61ms/step - loss: -6202850.0000 - accuracy: 0.1943 - val_loss: -8660338.0000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 9s 45ms/step - loss: -13321475.0000 - accuracy: 0.1943 - val_loss: -17326170.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 60m

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + Speaker
# Concatenate 'statement' and 'speaker' columns and provide it as input
def getHybridXY1(data): 
  y = data['label'].values
  X = data['statement'].values+data['speaker'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)


Epoch 1/30
190/190 [==============================] - 16s 77ms/step - loss: -1558.3479 - accuracy: 0.1943 - val_loss: -8235.5586 - val_accuracy: 0.1984
Epoch 2/30
190/190 [==============================] - 14s 73ms/step - loss: -65936.7422 - accuracy: 0.1943 - val_loss: -174187.7656 - val_accuracy: 0.1984
Epoch 3/30
190/190 [==============================] - 13s 70ms/step - loss: -514772.1250 - accuracy: 0.1943 - val_loss: -966493.6875 - val_accuracy: 0.1984
Epoch 4/30
190/190 [==============================] - 13s 68ms/step - loss: -1985614.2500 - accuracy: 0.1943 - val_loss: -3105717.5000 - val_accuracy: 0.1984
Epoch 5/30
190/190 [==============================] - 10s 55ms/step - loss: -5315688.5000 - accuracy: 0.1943 - val_loss: -7430963.0000 - val_accuracy: 0.1984
Epoch 6/30
190/190 [==============================] - 13s 66ms/step - loss: -11490533.0000 - accuracy: 0.1943 - val_loss: -14950065.0000 - val_accuracy: 0.1984
Epoch 7/30
190/190 [==============================] - 13s 67m

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + Subject
# Concatenate 'statement' and 'subject' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['subject'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)


Epoch 1/10
190/190 [==============================] - 13s 66ms/step - loss: -1123.4097 - accuracy: 0.1938 - val_loss: -6302.8076 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 11s 60ms/step - loss: -56889.8555 - accuracy: 0.1943 - val_loss: -150741.5625 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 14s 74ms/step - loss: -452400.5625 - accuracy: 0.1943 - val_loss: -851753.3750 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 11s 59ms/step - loss: -1755536.7500 - accuracy: 0.1943 - val_loss: -2750540.0000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 65ms/step - loss: -4730885.5000 - accuracy: 0.1943 - val_loss: -6651749.5000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 63ms/step - loss: -10267682.0000 - accuracy: 0.1943 - val_loss: -13393136.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 10s 50m

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + speaker job title
# Concatenate 'statement' and 'speaker job title' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['speaker job title'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)


Epoch 1/10
190/190 [==============================] - 14s 66ms/step - loss: -1061.1967 - accuracy: 0.1936 - val_loss: -5418.8311 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 10s 52ms/step - loss: -40651.2070 - accuracy: 0.1943 - val_loss: -104001.8984 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 14s 72ms/step - loss: -301070.6875 - accuracy: 0.1943 - val_loss: -554392.4375 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 13s 66ms/step - loss: -1122751.0000 - accuracy: 0.1943 - val_loss: -1737727.0000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 64ms/step - loss: -2975887.5000 - accuracy: 0.1943 - val_loss: -4133655.7500 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 11s 56ms/step - loss: -6388657.0000 - accuracy: 0.1943 - val_loss: -8275629.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 60ms/

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + state info
# Concatenate 'statement' and 'state info' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['state info'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)


Epoch 1/10
190/190 [==============================] - 12s 58ms/step - loss: -1698.0819 - accuracy: 0.1940 - val_loss: -8577.8633 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 15s 79ms/step - loss: -63744.8320 - accuracy: 0.1943 - val_loss: -162853.1406 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 12s 64ms/step - loss: -465517.1562 - accuracy: 0.1943 - val_loss: -859246.0000 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 10s 53ms/step - loss: -1738582.8750 - accuracy: 0.1943 - val_loss: -2691579.5000 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 12s 66ms/step - loss: -4581034.0000 - accuracy: 0.1943 - val_loss: -6388207.5000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 65ms/step - loss: -9790500.0000 - accuracy: 0.1943 - val_loss: -12719079.0000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 11s 59ms

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + party affilation
# Concatenate 'statement' and 'party affilation' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['party affilation'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y


# Creating a tokenizer to convert text to sequences
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)


Epoch 1/10
190/190 [==============================] - 13s 63ms/step - loss: -953.2339 - accuracy: 0.1935 - val_loss: -4612.2568 - val_accuracy: 0.1984
Epoch 2/10
190/190 [==============================] - 11s 57ms/step - loss: -31876.7656 - accuracy: 0.1943 - val_loss: -80221.2656 - val_accuracy: 0.1984
Epoch 3/10
190/190 [==============================] - 13s 69ms/step - loss: -225913.1719 - accuracy: 0.1943 - val_loss: -413576.8125 - val_accuracy: 0.1984
Epoch 4/10
190/190 [==============================] - 12s 62ms/step - loss: -836843.0000 - accuracy: 0.1943 - val_loss: -1290745.6250 - val_accuracy: 0.1984
Epoch 5/10
190/190 [==============================] - 9s 49ms/step - loss: -2190837.7500 - accuracy: 0.1943 - val_loss: -3043496.0000 - val_accuracy: 0.1984
Epoch 6/10
190/190 [==============================] - 12s 62ms/step - loss: -4656988.5000 - accuracy: 0.1943 - val_loss: -6032833.5000 - val_accuracy: 0.1984
Epoch 7/10
190/190 [==============================] - 12s 65ms/step

In [ ]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

In [ ]:
# For Hybrid Model : Statement + location of statement
# Concatenate 'statement' and 'location of statement' columns and provide it as input
def getHybridXY(data): 
  y = data['label'].values
  X = data['statement'].values+data['location of statement'].values
  for i in range(len(X)):
    X[i] = preprocess(X[i])
  return X,y

  # Preprocessing text data
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Creating a tokenizer to convert text to sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Converting text to sequences using the tokenizer
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Padding sequences to a fixed length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Building the CNN model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', fi_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
# test_preds = np.round(classifier.predict(test_seq))
# print(classification_report(y_test, test_preds))


In [24]:
# Importing necessary libraries
import tensorflow as tf
import transformers
import numpy as np
from sklearn.metrics import classification_report



# Loading the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.TFAutoModel.from_pretrained(model_name)

# Preprocessing text data





Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [25]:
X_train, y_train = getHybridXY(train_df)
X_test, y_test = getHybridXY(test_df)

# Tokenizing the input data
max_length = 100

train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# Converting the input data to tensors
train_seq = tf.data.Dataset.from_tensor_slices((
    dict(train_tokens),
    y_train
)).batch(32)
test_seq = tf.data.Dataset.from_tensor_slices((
    dict(test_tokens),
    y_test
)).batch(32)

# Building a custom classification layer on top of the pre-trained BERT model
input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

embeddings = model.bert(input_ids, attention_mask=attention_mask)[1]
dense = tf.keras.layers.Dense(128, activation='relu')(embeddings)
dropout = tf.keras.layers.Dropout(0.3)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

In [27]:
classifier = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiling and training the model
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', f1_metric])

history = classifier.fit(
    train_seq,
    validation_data=test_seq,
    epochs=3,
    verbose=1
)

# Evaluating the model on the test set
_, acc = classifier.evaluate(test_seq)
print('Test accuracy:', acc)
test_preds = np.round(classifier.predict(test_seq))
print(classification_report(y_test, test_preds))

Epoch 1/3
 21/238 [=>............................] - ETA: 2:36:39 - loss: 0.0000e+00 - accuracy: 0.1949 - f1_metric: 0.9409

KeyboardInterrupt: ignored